In [1]:
import pandas as pd
import numpy as np

In [145]:
from sklearn import preprocessing

In [202]:
from sklearn.metrics import precision_score

In [204]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [293]:
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report,confusion_matrix
import sklearn.model_selection as model_selection

from random import sample

from sklearn import preprocessing

from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer

from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV


### Column Description

● store_id - The unique identifier of a store.

● product_id - The unique identifier of a product.

● date - Sales date (YYYY-MM-DD)

● sales - Sales quantity

● revenue - Daily total sales revenue

● stock - End of day stock quantity

● price - Product sales price

● promo_type_1 - Type of promotion applied on channel 1

● promo_bin_1 - Binned promotion rate for applied promo_type_1

● promo_type_2 - Type of promotion applied on channel 2

● promo_bin_2 - Binned promotion rate for applied promo_type_2

● promo_discount_2 - Discount rate for applied promo type 2

● promo_discount_type_2 - Type of discount applied

● product_length - Length of product

● product_depth - Depth of product

● product_width - Width of product

● hierarchy1_id

● hierarchy2_id

● hierarchy3_id

● hierarchy4_id

● hierarchy5_id

● storetype_id

● store_size

● city_id

● train_or_test - rows with train tag will be used to train KNNRegressor and rows with test
tag will be used for accuracy calculation

## Basic EDA

In [318]:
sales = pd.read_csv("sales.csv")
ph = pd.read_csv("product_hierarchy.csv")
store_cities = pd.read_csv("store_cities.csv")

In [319]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [320]:
print("Sales size: ", sales.shape)
print("Product Hierarchy size: ", ph.shape)
print("Store Cities size: ", store_cities.shape)

Sales size:  (19454838, 13)
Product Hierarchy size:  (699, 10)
Store Cities size:  (144, 4)


In [321]:
sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [322]:
sales.isna().sum()

product_id                      0
store_id                        0
date                            0
sales                     1190762
revenue                   1190762
stock                     1190762
price                      571981
promo_type_1                    0
promo_bin_1              16803149
promo_type_2                    0
promo_bin_2              19395757
promo_discount_2         19395757
promo_discount_type_2    19395757
dtype: int64

In [323]:
print("Min Date: ", sales.date.min() )
print("Max Date: ", sales.date.max() )

Min Date:  2017-01-02
Max Date:  2019-12-29


In [324]:
ph.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [325]:
store_cities.head()

,store_id,storetype_id,store_size,city_id
0,S0091,ST04,19,C013
1,S0012,ST04,28,C005
2,S0045,ST04,17,C008
3,S0032,ST03,14,C019
4,S0027,ST04,24,C022


In [229]:
q2_sales = pd.read_csv("Q2/Q2_sales.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [230]:
q2_sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0081,S0065,2017-01-02,0.0,0.00,4.0,48.00,PR14,NaN,PR03,NaN,NaN,NaN
1,P0081,S0090,2017-01-02,0.0,0.00,5.0,48.00,PR14,NaN,PR03,NaN,NaN,NaN
2,P0081,S0095,2017-01-02,1.0,44.44,5.0,48.00,PR14,NaN,PR03,NaN,NaN,NaN
3,P0106,S0001,2017-01-02,0.0,0.00,2.0,18.95,PR05,low,PR03,NaN,NaN,NaN
4,P0106,S0002,2017-01-02,0.0,0.00,18.0,18.95,PR05,low,PR03,NaN,NaN,NaN


In [231]:
ph.head(20)

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917
5,P0006,8.5,15.0,15.0,cluster_0,H03,H0316,H031608,H03160817,H0316081708
6,P0007,2.0,22.0,9.5,cluster_4,H03,H0313,H031305,H03130519,H0313051904
7,P0008,5.0,16.0,5.0,cluster_0,H00,H0000,H000004,H00000400,H0000040017
8,P0009,5.0,18.0,14.0,cluster_6,H00,H0002,H000201,H00020100,H0002010012
9,P0010,2.0,22.0,3.0,cluster_0,H01,H0108,H010801,H01080109,H0108010917


In [31]:
ph[ph.hierarchy5_id=="H0001021017"]

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
133,P0141,6.0,20.0,12.0,cluster_6,H00,H0001,H000102,H00010210,H0001021017
157,P0166,5.0,19.0,13.0,cluster_0,H00,H0001,H000102,H00010210,H0001021017
256,P0272,7.4,19.5,12.9,cluster_0,H00,H0001,H000102,H00010210,H0001021017
272,P0290,5.0,19.0,13.0,cluster_0,H00,H0001,H000102,H00010210,H0001021017
285,P0303,6.0,17.0,10.0,NaN,H00,H0001,H000102,H00010210,H0001021017
543,P0581,3.5,19.0,14.5,cluster_6,H00,H0001,H000102,H00010210,H0001021017
631,P0675,6.5,26.0,13.0,cluster_0,H00,H0001,H000102,H00010210,H0001021017


In [51]:
ph.hierarchy5_id.iloc[1][:9]

'H01050100'

In [54]:
# PH Doğrusal mı
for i in range(len(ph)):
    if not ph.hierarchy5_id.iloc[i][:9] == ph.hierarchy4_id.iloc[i]:
        print("eş değil")

In [55]:
ph.groupby(["hierarchy5_id"]).count().head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id
hierarchy5_id,,,,,,,,,
H0000030001,5,5,5,5,5,5,5,5,5
H0000030901,1,1,1,1,1,1,1,1,1
H0000031001,1,1,1,1,1,1,1,1,1
H0000040001,5,5,5,5,4,5,5,5,5
H0000040012,3,3,3,3,2,3,3,3,3


In [19]:
print("Q2 Sales size: ", q2_sales.shape)

Q2 Sales size:  (1480193, 13)


In [18]:
q2_sales.isna().sum()

product_id                     0
store_id                       0
date                           0
sales                          0
revenue                        0
stock                          0
price                      36574
promo_type_1                   0
promo_bin_1              1267540
promo_type_2                   0
promo_bin_2              1478408
promo_discount_2         1478408
promo_discount_type_2    1478408
dtype: int64

### Q3 - Classification #2

When there is a new product in the system, every business user has a business experience to guess what will be the seasonality curve in the future. Can we replace a business user's experience with an algorithm?

In [326]:
q3_submissions = pd.read_csv("Q3/Q3_submission.csv")

In [327]:
predictions= list(q3_submissions.product_id.values)

In [328]:
sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [329]:
ph.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [334]:
sales_products = sales.merge(ph,on='product_id', how='inner')

In [335]:
sales_products.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006


In [336]:
print("Unique Products in ph: ",len(ph.product_id.unique()))
print("Unique Products in sales: ",len(sales.product_id.unique()))

Unique Products in ph:  699
Unique Products in sales:  649


In [337]:
q3_train = ph[~ph.product_id.isin(predictions)]
q3_submission = ph[ph.product_id.isin(predictions)]

In [338]:
q3_train.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917
5,P0006,8.5,15.0,15.0,cluster_0,H03,H0316,H031608,H03160817,H0316081708


In [339]:
q3_submission.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
12,P0013,7.1,27.5,13.7,NaN,H01,H0105,H010503,H01050309,H0105030906
22,P0023,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536
38,P0040,7.0,26.5,7.0,NaN,H03,H0312,H031204,H03120400,H0312040000
53,P0056,NaN,12.0,10.0,NaN,H03,H0314,H031401,H03140102,H0314010203


In [340]:
q3_train.sort_values(by="cluster_id",inplace=True)
q3_train.index = q3_train.product_id
q3_train.drop("product_id",axis=1, inplace=True)

q3_submission.index = q3_submission.product_id
q3_submission.drop("product_id",axis=1, inplace=True)

In [341]:
q3_train.drop(q3_train.columns[4:8],axis=1,inplace=True)
q3_submission.drop(q3_submission.columns[4:8],axis=1,inplace=True)

In [342]:
q3_train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id
product_id,,,,,
P0376,2.0,13.0,8.0,cluster_0,H0001051017
P0470,7.0,18.0,19.0,cluster_0,H0002000026
P0469,3.8,21.6,35.0,cluster_0,H0313021001
P0468,5.0,42.0,12.0,cluster_0,H0315020911
P0467,2.5,17.0,7.0,cluster_0,H0315041121


In [343]:
q3_submission.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id
product_id,,,,,
P0000,5.0,20.0,12.0,NaN,H0004010534
P0013,7.1,27.5,13.7,NaN,H0105030906
P0023,1.0,1.0,1.0,NaN,H0004030536
P0040,7.0,26.5,7.0,NaN,H0312040000
P0056,NaN,12.0,10.0,NaN,H0314010203


In [344]:
q3_train.cluster_id.value_counts()

cluster_0    450
cluster_9     51
cluster_4     36
cluster_3     31
cluster_6     28
cluster_8     14
cluster_5     13
cluster_2     12
cluster_7     10
cluster_1      4
Name: cluster_id, dtype: int64

**Label Encoding**

In [345]:
le = preprocessing.LabelEncoder()

In [346]:
le.fit(q3_train.cluster_id)

LabelEncoder()

In [347]:
q3_train.cluster_id = le.transform(q3_train.cluster_id)

In [348]:
le = preprocessing.LabelEncoder()
le.fit(q3_train.hierarchy5_id)
q3_train.hierarchy5_id = le.transform(q3_train.hierarchy5_id)

In [349]:
q3_train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id
product_id,,,,,
P0376,2.0,13.0,8.0,0,27
P0470,7.0,18.0,19.0,0,33
P0469,3.8,21.6,35.0,0,265
P0468,5.0,42.0,12.0,0,332
P0467,2.5,17.0,7.0,0,336


In [350]:
le = preprocessing.LabelEncoder()
le.fit(q3_submission.hierarchy5_id)
q3_submission.hierarchy5_id = le.transform(q3_submission.hierarchy5_id)

In [354]:
q3_submission.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id
product_id,,,,,
P0000,5.0,20.0,12.0,NaN,14
P0013,7.1,27.5,13.7,NaN,18
P0023,1.0,1.0,1.0,NaN,16
P0040,7.0,26.5,7.0,NaN,38
P0056,NaN,12.0,10.0,NaN,44


**Model without sales**

In [362]:
q3_train.product_length.fillna(q3_train.product_length.median(),inplace=True)

In [363]:
q3_train.product_depth.fillna(q3_train.product_depth.median(),inplace=True)

In [364]:
q3_train.product_width.fillna(q3_train.product_width.median(),inplace=True)

In [365]:
q3_submission.product_length.fillna(q3_train.product_length.median(),inplace=True)

In [366]:
q3_submission.product_depth.fillna(q3_train.product_depth.median(),inplace=True)

In [367]:
q3_submission.product_width.fillna(q3_train.product_width.median(),inplace=True)

In [368]:
q3_train.isna().sum()

product_length    0
product_depth     0
product_width     0
cluster_id        0
hierarchy5_id     0
dtype: int64

In [369]:
q3_submission.isna().sum()

product_length     0
product_depth      0
product_width      0
cluster_id        50
hierarchy5_id      0
dtype: int64

In [373]:
X_train = q3_train.drop("cluster_id",axis=1)[:500]
y_train = q3_train.cluster_id[:500].values
X_test = q3_submission.drop("cluster_id",axis=1)
y_test = q3_submission.cluster_id.values

In [374]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [375]:
rfc_pred = classifier.predict(X_test)

In [378]:
print ("Random Forest Train Accuracy Baseline:", metrics.accuracy_score(y_train, classifier.predict(X_train)))

Random Forest Train Accuracy Baseline: 0.984


In [379]:
rfc_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0])

In [398]:
for i in range(len(rfc_pred)):
    q3_submissions.cluster_id.iloc[i] = "cluster_"+str(rfc_pred[i])

In [399]:
q3_submissions.head()

,product_id,cluster_id
0,P0000,cluster_0
1,P0013,cluster_0
2,P0023,cluster_0
3,P0040,cluster_0
4,P0056,cluster_0


In [400]:
q3_submissions.to_csv("q3_sub.csv",index=False)

**Merging with Sales**

In [253]:
sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [254]:
sales_sum = sales.groupby("product_id").sum()

In [255]:
sales_sum.head()

,sales,revenue,stock,price,promo_discount_2
product_id,,,,,
P0001,2969.0,21128.44,156616.0,190086.55,0.0
P0002,48.0,9776.39,27380.0,658550.00,0.0
P0004,423.0,1631.29,27534.0,9486.50,0.0
P0005,714.0,25408.62,161979.0,1281136.30,6440.0
P0006,113.0,2550.17,13030.0,84502.40,0.0


In [256]:
products_with_sales = pd.concat([q3_train, sales_sum], axis=1)

In [257]:
products_with_sales.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id,sales,revenue,stock,price,promo_discount_2
P0376,2.0,13.0,8.0,0,27,5107.0,28739.17,262204.0,210715.60,0.0
P0470,7.0,18.0,19.0,0,33,7101.0,147696.59,420510.0,1838440.55,27380.0
P0469,3.8,21.6,35.0,0,265,155.0,14069.45,62461.0,1086829.50,0.0
P0468,5.0,42.0,12.0,0,332,52.0,1249.52,7468.0,53906.74,0.0
P0467,2.5,17.0,7.0,0,336,23.0,724.29,3712.0,43521.84,560.0


In [258]:
print("products_with_sales uzunluğu: ", len(products_with_sales))

products_with_sales uzunluğu:  649


**Basic Model with products with sales data**

In [259]:
products_with_sales.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id,sales,revenue,stock,price,promo_discount_2
P0376,2.0,13.0,8.0,0,27,5107.0,28739.17,262204.0,210715.60,0.0
P0470,7.0,18.0,19.0,0,33,7101.0,147696.59,420510.0,1838440.55,27380.0
P0469,3.8,21.6,35.0,0,265,155.0,14069.45,62461.0,1086829.50,0.0
P0468,5.0,42.0,12.0,0,332,52.0,1249.52,7468.0,53906.74,0.0
P0467,2.5,17.0,7.0,0,336,23.0,724.29,3712.0,43521.84,560.0


In [264]:
products_with_sales.product_length.fillna(products_with_sales.product_length.median(),inplace=True)

In [265]:
products_with_sales.product_depth.fillna(products_with_sales.product_depth.median(),inplace=True)

In [266]:
products_with_sales.product_width.fillna(products_with_sales.product_width.median(),inplace=True)

In [267]:
products_with_sales.isna().sum()

product_length      0
product_depth       0
product_width       0
cluster_id          0
hierarchy5_id       0
sales               0
revenue             0
stock               0
price               0
promo_discount_2    0
dtype: int64

In [268]:
# Shuffle the data, because it's not date based.
products_with_sales = products_with_sales.sample(frac=1)

In [269]:
products_with_sales.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id,sales,revenue,stock,price,promo_discount_2
P0616,6.0,9.5,7.5,0,55,379.000,8061.96,26142.000,123762.55,560.0
P0527,4.5,20.0,15.5,0,18,32640.000,64061.49,1316483.000,201036.74,14200.0
P0298,5.0,17.0,11.0,9,199,376.898,1333.60,7392.698,2104.69,0.0
P0210,5.2,33.5,12.0,7,236,3997.000,42886.01,303080.000,435284.70,0.0
P0494,10.0,14.0,11.0,0,244,118.000,1579.02,12955.000,12486.20,0.0


In [270]:
X_train = products_with_sales.drop("cluster_id",axis=1)[:500]
y_train = products_with_sales.cluster_id[:500].values
X_test = products_with_sales.drop("cluster_id",axis=1)[500:]
y_test = products_with_sales.cluster_id[500:].values

In [271]:
# Feature Scaling
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [272]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(scaled_X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [288]:
rfc_pred = classifier.predict(X_test)

In [297]:
print ("Random Forest Train Accuracy Baseline:", metrics.accuracy_score(y_train, classifier.predict(scaled_X_train)))
print ("Random Forest Test Accuracy Baseline:", metrics.accuracy_score(y_test, classifier.predict(scaled_X_test)))
#print('Accuracy of random forest classifier on test set: {:.2f}'.format(rfc.score(X_test, y_test)))

Random Forest Train Accuracy Baseline: 0.852
Random Forest Test Accuracy Baseline: 0.6845637583892618


In [298]:
print(confusion_matrix(y_test,rfc_pred))

[[104   0   0   0   0   0   0   0   0   1]
 [ 16   0   0   0   0   0   0   0   0   0]
 [  2   0   0   0   0   0   0   0   0   0]
 [  7   0   0   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0]
 [  2   0   0   0   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0   0   0]]


### Submission

In [301]:
products_with_sales.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id,sales,revenue,stock,price,promo_discount_2
P0616,6.0,9.5,7.5,0,55,379.000,8061.96,26142.000,123762.55,560.0
P0527,4.5,20.0,15.5,0,18,32640.000,64061.49,1316483.000,201036.74,14200.0
P0298,5.0,17.0,11.0,1,199,376.898,1333.60,7392.698,2104.69,0.0
P0210,5.2,33.5,12.0,2,236,3997.000,42886.01,303080.000,435284.70,0.0
P0494,10.0,14.0,11.0,0,244,118.000,1579.02,12955.000,12486.20,0.0


In [302]:
q3_submission.head()

,product_length,product_depth,product_width,cluster_id,hierarchy5_id
product_id,,,,,
P0000,5.0,20.0,12.0,NaN,14
P0013,7.1,27.5,13.7,NaN,18
P0023,1.0,1.0,1.0,NaN,16
P0040,7.0,26.5,7.0,NaN,38
P0056,NaN,12.0,10.0,NaN,44


In [303]:
len(q3_submission)

50

In [311]:
q3_submission.product_length.fillna(q3_submission.product_length.median(),inplace=True)

In [312]:
q3_submission.product_depth.fillna(q3_submission.product_depth.median(),inplace=True)

In [313]:
q3_submission.product_width.fillna(q3_submission.product_width.median(),inplace=True)

In [314]:
q3_submission.isna().sum()

product_length     0
product_depth      0
product_width      0
cluster_id        50
hierarchy5_id      0
dtype: int64

In [315]:
X_train = products_with_sales.drop("cluster_id",axis=1)
y_train = products_with_sales.cluster_id.values
X_test = q3_submission.drop("cluster_id",axis=1)

In [316]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=15, random_state=42)

In [300]:
q3_submissions.head()

,product_id,cluster_id
0,P0000,NaN
1,P0013,NaN
2,P0023,NaN
3,P0040,NaN
4,P0056,NaN
